## CLAP implementation from HuggingFace
Zeroshot audio classification

Need to install, datasets, transformers, either tensorflow or pytorch, soundfile, librosa

In [1]:
from transformers import pipeline
from datasets import load_dataset
from IPython.display import Audio

/home/trips/Virtual_env_python/lib64/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'datasets'

In [58]:
# details about this particualr dataset available at https://github.com/suzuki256/dog-dataset
dog_barks = load_dataset("437aewuh/dog-dataset", split="train", streaming=True) # dataset of dogbarks. 3 classes: adult, puppy, mix(general dog)

Resolving data files:   0%|          | 0/300 [00:00<?, ?it/s]

In [96]:
dog_barks = dog_barks.shuffle(seed=42, buffer_size=100)  #shuffles the iterable dataset
sample = next(iter(dog_barks))   # selects the first row from the dog_barks 
audio_sample_array = sample["audio"]["array"] # numberical array of the selected sample

In [97]:
# checking the id and verifying the sound
print(sample)
Audio(audio_sample_array, rate = 44100)

{'audio': {'path': 'hf://datasets/437aewuh/dog-dataset@73810687d1afab5e8318f4d7e51bfd7175d1848d/adult_dog/adult_dog_0014.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00198364,
       -0.00170898, -0.0017395 ]), 'sampling_rate': 44100}, 'label': 0}


In [107]:
# Different candidate labels for the zero shot evaluation
candidate_labels = ["Sound of a puppy", "Sound of an adult dog","Sound of a dog"]
candidate_labels1 = ["Sound of an toy dog", "Sound of real dog"]
candidate_labels2 = ["Sound of an adult dog", "Sound of a puppy"]

## Loading CLAP model

In [103]:
classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")

In [108]:
classifier(audio_sample_array, candidate_labels=candidate_labels)

[{'score': 0.6315874457359314, 'label': 'Sound of a dog'},
 {'score': 0.3127667307853699, 'label': 'Sound of an adult dog'},
 {'score': 0.055645886808633804, 'label': 'Sound of a puppy'}]

In [109]:
classifier(audio_sample_array, candidate_labels=candidate_labels1)

[{'score': 0.9551675319671631, 'label': 'Sound of real dog'},
 {'score': 0.044832486659288406, 'label': 'Sound of an toy dog'}]

In [110]:
classifier(audio_sample_array, candidate_labels=candidate_labels2)

[{'score': 0.8489577174186707, 'label': 'Sound of an adult dog'},
 {'score': 0.15104229748249054, 'label': 'Sound of a puppy'}]